# Policy Gradient Methods

Policy _based_ methods learn the optimal policy directly, without necessarily estimating a value
function. Policy _gradient_ methods do that performing gradient ascent on the objective function.

### Advantages

- No need to store action-values.
- Ability to learn a stochastic policy direcly.
- Hence, no need to manually tune exploitation vs. exploration.
- Effective in continuous action spaces (and high-dimensional state spaces).
- They generally have good convergence properties.

### Disadvantages

- They might have high-variance.
- Might converge to a local maximum.
- Slower than other methods, and might take a long time to train.


In [ ]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from typing import Union

import gymnasium as gym

from util.gymnastics import gym_simulation, init_random, plot_scores

## Cart Pole Environment

Explore the
[Gymnasium Cart Pole environment](https://gymnasium.farama.org/environments/classic_control/cart_pole/),
and run the simulation below!

**IMPORTANT**: For this notebook, we are going to tweak the reward function of the environment. See
the "Reward Function" section.


In [ ]:
# `gym_simulation` is a convenient utility to run simulations on Gynmasium environment, so that we
# don't have to repeat the same code in all notebooks. But it works basically the same as the one we
# implemented for DQN :) Feel free to check the source code!

gym_simulation("CartPole-v1")

In [ ]:
# Just for convenience, we hardcode the state and action sizes of the CartPole environment.
STATE_SIZE = 4
ACTION_SIZE = 2

### Reward Function

The Cart Pole reward function returns `+1` reward at every timestep: the idea is that the longer the
pole stays in the upright position the better. Such reward function does not encapsulate whether an
action is good or bad, and it does not play well with some concepts we are going to analyze later in
this notebook (such as normalization and custom baselines).

For this reason, we "adjust" the reward taking into account angle and position of the pole as a
heuristic to evaluate how good is the next state (to which the agent transitioned given its action).


In [ ]:
# This function works for both single and vectorized environments and rewards.
def adjust_reward(next_state, reward) -> Union[np.float64, np.array]:
    angle = next_state[2] if len(next_state.shape) == 1 else next_state[:, 2]
    position = next_state[0] if len(next_state.shape) == 1 else next_state[:, 0]
    return reward - np.abs(angle) / 0.418 - np.abs(position) / 4.8

## Optimization Rule

For one trajectory $\tau$ (or episode), the neural networks weight can be updated according to:

$$
\theta_{k+1} = \theta_k - \alpha \sum_{t=0} \nabla_{\theta} \log \pi_{\theta}(a_t|s_t) R(\tau)
$$

We can interpret this as pushing up probabilities for action / states combinations when the return
is high, and the other way around for low returns.

This relationship is also interesting because only the policy function needs to be differentiable:
the reward function might very well be discontinuous and sparse.

For derivation, check the
[Hugging Face Deep RL tutorial](https://huggingface.co/learn/deep-rl-course/unit4/pg-theorem).


## REINFORCE

<div style="width: 70%">
  <img src="assets/04_PG_reinforce.png">
  <br>
  <small>Sutton & Barto 2022</small>
</div>


In [ ]:
class PolicyNetwork(nn.Module):
    def __init__(self, hidden_units=16):
        super(PolicyNetwork, self).__init__()
        # TODO: Create two fully connected / linear layers, input dimension STATE_SIZE, output
        #       dimension ACTION_SIZE, and hidden units specified in the constructor.

    def forward(self, x):
        # TODO: Use ReLU as first non-linearity, and softmax for the output layer (so that we can
        #       interpret this as a stochastic policy across action probabilities). Hint: make sure
        #       to choose the right dimension for the softmax! Another hint: it should be the last
        #       dimension of the tensor (to be able to work with single and batched updates).
        pass

In [ ]:
# Agent that will work with both single and vectorized environments, keep that in mind when thinking
# about dimensionality and how to operate on tensors in a consistent way!
class Agent:
    def __init__(self):
        self.policy = PolicyNetwork()
        self.optimizer = optim.Adam(self.policy.parameters(), lr=1e-2)

    def sample_action(self, state: np.array) -> tuple[np.array, torch.Tensor]:
        """Samples an action for a state from the policy network."""
        # TODO: Convert the state to a PyTorch tensor.
        state = None
        # TODO: Get the action probabilities from the policy.
        probs = None
        # TODO: Create a Categorical distribution with those probabilities.
        cdist = None
        # TODO: Sample the action from the categorical distribution, and gets its logprob.
        action = None
        logprob = None
        # TODO: Return the action and convert it to be used by Gymnasium (hint: use cpu().numpy()).
        #       Also return the the log_prob for that action as well as second return output.
        return None

    def learn(self, log_probs: list[torch.Tensor], returns: Union[np.float64, np.array]):
        """Perform a step of learning (gradient ascent) on the policy network."""
        # TODO: To handle both single and vectorized environments, rewards can be either a scalar or
        #       an array. Let's create a corresponding tensor first.
        returns = None
        # TODO: Stack the log_probs (hint: use torch.stack)
        log_probs = None
        # TODO: Compute the policy loss. Hint: gradient ascent.... so negative sign!
        policy_loss = None
        # TODO: Perform a backprop step via the optimizer.
        pass

    @torch.no_grad
    def act(self, state):
        """Convenient method for the agent to select an action during simulation."""
        action, _ = self.sample_action(state[np.newaxis, :])
        return action[0]

In [ ]:
def REINFORCE(env: gym.Env, agent, max_episodes=10_000, max_t=1_000, gamma=0.9999):
    # Tracks the score for each episode.
    #
    # IMPORTANT: in this notebook we modified the reward function but effectively we still consider
    # the score as the original reward function. In fact, we just track the latest timestamp of the
    # episode (representing how long we were able to balance the pole). This is important to know to
    # properly read the "score" value and its meaning (which still encapsulates learning quality).
    scores = []
    # Loop for max_episodes.
    for i_episode in range(1, max_episodes + 1):
        # Store episode rewards.
        rewards = []
        # Store episode log probabilities.
        log_probs = []
        # Start the episode in the initial state.
        state, _ = env.reset()

        # TODO: Generate an episode following the policy, for T (max_t) timesteps.
        for t in range(max_t):
            # TODO: sample action and log probability.
            action, log_prob = None
            # TODO: perform a step in the environment.
            state, reward, terminated, truncated, _ = None
            # Storing the reward (adjusting it for the purpose of this notebook).
            reward = adjust_reward(state, reward)
            rewards.append(adjust_reward(state, reward))
            # TODO: store reward and log probability.
            # TODO: Check for episode completion.

        # Compute discounted return.
        # TODO: First, compute the discounts.
        discounts = None
        # TODO: Then compute the total discounted return as the sum of the discouted rewards.
        R = None

        # TODO: Perform a learning step on the agent calling the `learn` method.

        # Track scores and print statistics.
        scores.append(t)
        avg_duration = np.mean(scores[-100:])
        if i_episode % 100 == 0:
            print(f"Episode {i_episode}\tAverage duration: {avg_duration:.2f}")
        if avg_duration >= 490.0:  # Solved
            print(f"Environment solved at episode {i_episode}\Avg. duration: {avg_duration:.2f}")
            break

    return scores

In [ ]:
with init_random(gym.make("CartPole-v1")) as env:
    agent = Agent()
    scores = REINFORCE(env, agent)
plot_scores(scores)

In [ ]:
gym_simulation("CartPole-v1", agent)

## Improvements

### Use Future Rewards

First thing to notice is that we are using all rewards at every timestep. But really, we should only
consider future rewards: i.e., the rewards that are actually the consequences of our actions.

$$
g = \sum_t R_t^{future}\nabla_{\theta} log\pi_{\theta}(a_t | s_t)
$$

### Collect Multiple Trajectories

In every episode, we sample a single trajectory: the gradient update might not contain good data
about our policy and the stochastic updates might be very _noisy_. Learning happens because in the
long run we hope that all tiny signals accumulate and converge towards the optimal policy.

How do we reduce noise? The simplest strategy is to collect more trajectories (with the current
policy) at once! And Gymnasium vectorized environment `gym.vector.VectorEnv` serves exactly that
purpose!

#### Normalize Rewards

When collecting multiple trajectories, a technique we can use is to normalize the rewards across the
various trajectories: which roughly picks half actions to encourage / discourage, and keeps the
gradient updates moderate.

$$
R_k \leftarrow \frac{R_k - \mu}{\sigma}
$$

The rewards distribution will also likely change during training, and a reward that might be good at
the beginning might actually signal a bad trajectory in late stages of training. Normalization helps
with such cases as well.

### Baseline Subtraction

The idea is to subtract to the reward a _baseline_ $b$, for example the average reward along all
trajectories (What if every trajectory has _always_ positive returns?). In this case, things that
are above average will push their probabilities to happen up while things below average will be
penalized.

$$
\theta_{k+1} = \theta_k - \alpha \sum_{t=0} \nabla_{\theta} \log \pi_{\theta}(a_t|s_t) [R(\tau) - b(s_t)]
$$

We can do this because on expectation this extra subtracted term will have zero effect (as long as
it does not depend on the action in _logprob_), but overall we'll get reduced variance (proof left
as exercise and/or you can find it in the resources).

#### Advantage Function

This value that we multiply with the log-probability to "reinforce" or "depress" the corresponding
actions is called the _advantage function_ and plays a critical role in state-of-the-art algorithms:

$$
A(*) = R(\tau) - b
$$

It measures how better the selected action does compared to the _average_ in the state.


In [ ]:
# Note how we use a vectorized environment: gym.vector.VectorEnv!
def REINFORCE_v2(
    env: gym.vector.VectorEnv,
    agent,
    max_episodes=10_000,
    max_t=1_000,
    gamma=0.9999,
    with_normalization=True,
    with_baseline=True,
):
    # Tracks the score for each episode. IMPORTANT: same remark as in REINFORCE above!
    scores = []
    # Loop for max_episodes.
    for i_episode in range(1, max_episodes + 1):
        # Store episode rewards, log probabilities, *AND* episode states for baseline computation!
        rewards, log_probs, states = ([], [], [])
        # Start the episode in the initial state.
        state, _ = env.reset()

        # TODO: Generate an episode following the policy. Copy the code above :) But...
        for t in range(max_t):
            # ...pay attention! We are now using a vectorized environment! Hence, to determine
            # episode completion we should check if `any()` is terminated or truncated!
            pass

        # Compute discounted _future_ returns.
        # TODO: Compute the discounts and discounted rewards.
        discounts = None
        # TODO: Pay attention to dimensionality of rewards (trajectory_len, n_envs), and how to
        #       broadcast discounts appropriately! Hint: use np.newaxis to add a dimension.
        discounted_rewards = None
        # TODO: Compute the future_returns. Hint: consider cumulative sums in reverse order :)
        #       But again, pay attention in which axis to perform the cumulative sum!
        future_returns = None

        # If the `with_baseline` argument flag is enabled, subtract a baseline to future returns.
        if with_baseline:
            # TODO: Compute the state-dependent baseline. Feel free to explore different baselines!
            #       As a suggestion, use the product of the angle and angular velocity: it provides
            #       a proxy about how well we are contrasting the pole fall.
            #       Hint: use the tracked `states` to compute the time dependent baseline.
            baseline = None
            # TODO: Subtract the baseline from future_returns.
            future_returns = None

        # If the `with_normalization` argument flag is enabled, normalize future returns.
        # Note: normalize across multiple trajectories of the vectorized env for the same timestamp.
        if with_normalization:
            # TODO: Compute the mean. Hint: use np.mean on the proper axis.
            returns_mean = None
            # TODO: Compute the stdev. Hint: make sure the std is non zero; use np.std.
            returns_std = None
            # TODO: Normalize the future returns using mean and stdev.
            future_returns = None

        # TODO: Perform a learning step calling agent.learn(...)
        # copy() for negative strides :(
        #   https://discuss.pytorch.org/t/negative-strides-in-tensor-error/134287/2

        # Track scores and print statistics
        scores.append(t)
        avg_duration = np.mean(scores[-100:])
        if i_episode % 100 == 0:
            print(f"Episode {i_episode}\tAverage duration: {avg_duration:.2f}")
        if avg_duration >= 490.0:  # Solved
            print(f"Environment solved at episode {i_episode}\tAvg. duration: {avg_duration:.2f}")
            break

    return scores

In [ ]:
env = init_random(gym.make_vec("CartPole-v1", num_envs=5))
agent_v2 = Agent()
scores_v2 = REINFORCE_v2(env, agent_v2)
plot_scores(scores_v2)

In [ ]:
gym_simulation("CartPole-v1", agent_v2)

## What Can We Do Better?

There are other improvements that can be applied:

1. Actor-critic setup (with value function baseline) and advanced advantage estimation such as _GAE_
   will improve learning.
2. We are currently _discarding experiences_ after every learning step. That is because the policy
   effectively changes. But we'll see that with importance sampling we can iterate on the same data
   multiple times and learn in mini-batches!
3. Techniques such as "_trust region_" and "_loss clipping_" will help against degeneration and keep
   the policy learning along smooth gradient directions.

Once we put all of these in place... we'll have PPO!


## Appendix

### Meaning of Loss

Note that the loss function used in policy gradient methods doesn't have the same meaning of the
typical supervised learning setup. In particular, after that first step of gradient descent, there
is no more connection to performance - which is determined by the average return.

The loss function is only useful when evaluated at the current parameters to perform one step of
gradient ascent. After that it loses its meaning and it's value shouldn't be used as a metric for
performance.

More details on
[OpenAI Spinning Up](https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html#id14).

### Performance Comparison

You can check how REINFORCE with future rewards + normalization + well crafted baseling performs
better than the vanilla REINFORCE.

Interesting to observe how we needed to tweak the reward function to reach these results: in fact,
formulating the reward function properly is one of the major challenges in RL and an area of open
research (e.g., curiosity driven agents).

As an exercise, try removing the adjustment of rewards in this notebook and just keep the original
Cart Pole `+1` per timestep reward. What's the effect on the algorithms' variants? Hint: check how
catastrophic learning is with normalization enabled!


In [ ]:
import matplotlib.pyplot as plt

init_random()
base, norm, all = ([], [], [])
random_seeds = np.random.randint(3_141_592, size=10)
for seed in random_seeds:
    env = init_random(gym.make_vec("CartPole-v1", num_envs=5), seed=int(seed))
    print("Future rewards only:")
    agent_v3 = Agent()
    scores_v3 = REINFORCE_v2(env, agent_v3, with_normalization=False, with_baseline=False)
    base.append(len(scores_v3))

    env = init_random(gym.make_vec("CartPole-v1", num_envs=5), seed=int(seed))
    print("Future rewards + normalization:")
    agent_v3_b = Agent()
    scores_v3_b = REINFORCE_v2(env, agent_v3_b, with_normalization=True, with_baseline=False)
    norm.append(len(scores_v3_b))

    env = init_random(gym.make_vec("CartPole-v1", num_envs=5), seed=int(seed))
    print("Future rewards + normalization + baseline:")
    agent_v3_bn = Agent()
    scores_v3_bn = REINFORCE_v2(env, agent_v3_bn, with_normalization=True, with_baseline=True)
    all.append(len(scores_v3_bn))
    print()

x = np.arange(len(norm))
plt.figure("Episode scores")
plt.plot(x, base, label="Future rewards")
plt.plot(x, norm, "r", label="Future rewards + normalization")
plt.plot(x, all, "g", label="Future rewards + normalization + baseline")
plt.ylabel("Score")
plt.xlabel("Episode #")
plt.show()